<a href="https://colab.research.google.com/github/UtkarshAIITB/IPL-Data-Analysis/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [305]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [306]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [307]:
new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/new.csv')
# new.head()

In [308]:
t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/t.csv')
# t.head()

In [309]:
t.drop(columns = ['Unnamed: 0'], inplace = True)
t.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,597998,2013-04-03,9,6,9,9,1,0.1,6,9,2013,1,0,0,128,0,1
1,597998,2013-04-03,9,6,9,9,1,0.2,6,9,2013,1,0,0,128,4,1
2,597998,2013-04-03,9,6,9,9,1,0.3,6,9,2013,1,0,0,128,5,1
3,597998,2013-04-03,9,6,9,9,1,0.4,6,9,2013,1,0,0,128,5,1
4,597998,2013-04-03,9,6,9,9,1,0.5,6,9,2013,1,0,0,128,5,1


In [310]:
new.drop(columns = ['Unnamed: 0'], inplace = True)

In [311]:
new.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,9,5,2008,1,0,0,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,9,5,2008,1,0,0,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,9,5,2008,1,0,0,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,9,5,2008,1,0,0,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,9,5,2008,1,0,0,222,2,0


In [312]:
# new is the training set and t is the test set

In [313]:
tnew = t.drop(t[(t.over <= 11.0) | (t.over >=16.0)].index)
tnew.reset_index(drop = True)
y1test = tnew['total_runs_y']
y2test = tnew['winner']
tnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
tnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,9,6,9,1,11.1,6,9,1,0,0,76,3
1,9,6,9,1,11.2,6,9,1,0,0,77,3
2,9,6,9,1,11.3,6,9,1,0,0,78,3
3,9,6,9,1,11.4,6,9,1,0,0,79,3
4,9,6,9,1,11.5,6,9,1,0,0,79,3


In [314]:
xnew = new.drop(new[(new.over <=11.0) | (new.over >=16.0)].index)
xnew.reset_index(drop = True)
y1train = xnew['total_runs_y']
y2train = xnew['winner']
xnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
xnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,5,9,5,1,11.1,9,5,1,0,0,102,1
1,5,9,5,1,11.2,9,5,1,0,0,108,1
2,5,9,5,1,11.3,9,5,1,0,0,110,1
3,5,9,5,1,11.4,9,5,1,0,0,111,1
4,5,9,5,1,11.5,9,5,1,0,0,111,1


In [315]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [316]:
xnewt = sc.fit_transform(xnew) 
tnewt = sc.fit_transform(tnew)

In [317]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [318]:
model = Sequential()
model.add(Dense(200, activation = 'relu'))

model.add(Dense(100, activation = 'relu'))

model.add(Dense(50, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [319]:
model.fit(x=xnewt, y=y1train, epochs=85, 
          validation_data=(tnewt,y1test))

Epoch 1/85
1370/1370 [==============================] - 3s 2ms/step - loss: 2300.6575 - val_loss: 493.1058
Epoch 2/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1339.1462 - val_loss: 458.7790
Epoch 3/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1305.7759 - val_loss: 421.2831
Epoch 4/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1276.7830 - val_loss: 438.3396
Epoch 5/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1257.5376 - val_loss: 403.9370
Epoch 6/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1272.3766 - val_loss: 460.8571
Epoch 7/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1237.3823 - val_loss: 436.3923
Epoch 8/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1224.0894 - val_loss: 457.8087
Epoch 9/85
1370/1370 [==============================] - 3s 2ms/step - loss: 1191.5717 - val_loss: 420.6360
Epoch 10/85
1370/1370 [==============

In [320]:
y1test = y1test.ravel()

In [321]:
predictions = model.predict(tnewt)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=y1test
sample.head(10)

,Predict,Actual
0,167.842896,128
1,168.295593,128
2,168.818329,128
3,169.370911,128
4,170.785858,128
5,120.663841,128
6,125.332016,128
7,126.629501,128
8,128.667358,128
9,128.012131,128


In [322]:
def custom_accuracy(y_test,y_pred,thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right += 1
    return ((right/l)*100)

In [323]:
custom_accuracy(sample['Actual'] , sample['Predict'],10)

37.45629370629371

In [324]:
# sample.to_csv('sample.csv')

In [325]:
# try1 = sample[sample.Actual == 128]
# try1.head()

In [326]:
# try1.shape[0]

In [327]:
# try1.sum()

In [328]:
# (try1.sum())/try1.shape[0]

In [329]:
# try2 = sample[sample.Actual == 129]
# (try2.sum())/try2.shape[0]

In [330]:
# try2 = sample[sample.Actual == 156]
# result = (try2.sum())/try2.shape[0]

In [331]:
# result

In [332]:
# res2 = result.values.reshape(1,2)

In [333]:
# res2

In [334]:
# res2.shape

In [335]:
# df.loc[len(df)] = your_array
# results.loc[len(results)] = res2
# df.append(pd.DataFrame(arr).T)
# results = results.append(pd.DataFrame(res2))
# results.append(pd.DataFrame(res3))

In [336]:
ar1 = sample['Actual'].unique()

In [337]:
results = pd.DataFrame()

In [338]:
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [339]:
results.head()

,0,1
0,135.215679,128.0
0,161.784148,129.0
0,160.469238,156.0
0,162.794965,154.0
0,169.546517,126.0


In [340]:
# rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)

In [341]:
results.reset_index(drop = True)

,Predict,Actual
0,135.215679,128.0
1,161.784148,129.0
2,160.469238,156.0
3,162.794965,154.0
4,169.546517,126.0
...,...,...
79,195.783264,190.0
80,141.693706,113.0
81,168.896264,172.0
82,149.095762,134.0


In [342]:
results.tail()

,Predict,Actual
0,195.783264,190.0
0,141.693706,113.0
0,168.896264,172.0
0,149.095762,134.0
0,178.338509,192.0


In [343]:
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

50.0


In [344]:
# results.info()

In [345]:
# results['Actual']

take epochs = 90 next time

In [346]:
tests = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/testset.csv')
# tests.head()
tests.drop(columns = ['Unnamed: 0'], inplace = True)
tests.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.1,12.0,5.0,1,0,0,159,2,0
1,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.2,12.0,5.0,1,0,0,159,2,0
2,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.3,12.0,5.0,1,0,0,159,2,0
3,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.4,12.0,5.0,1,0,0,159,4,0
4,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.5,12.0,5.0,1,0,0,159,4,0


In [347]:
sets = tests.drop(tests[(tests.over <= 11.0) | (tests.over >=16.0)].index)
sets.reset_index(drop = True)
yt1 = sets['total_runs_y']
yt2 = sets['winner']
sets.drop(columns = ['id', 'date', 'total_runs_y', 'winner'], inplace = True)
sets.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,12.0,5.0,5.0,1,11.1,12.0,5.0,1,0,0,95,2
1,12.0,5.0,5.0,1,11.2,12.0,5.0,1,0,0,95,2
2,12.0,5.0,5.0,1,11.3,12.0,5.0,1,0,0,96,2
3,12.0,5.0,5.0,1,11.4,12.0,5.0,1,0,0,96,2
4,12.0,5.0,5.0,1,11.5,12.0,5.0,1,0,0,97,2


In [348]:
yt1 = yt1.ravel()

In [349]:
xs = sc.fit_transform(sets)

In [350]:
predictions = model.predict(xs)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=yt1
sample.head(10)

,Predict,Actual
0,174.904099,159
1,173.492065,159
2,173.514313,159
3,172.106171,159
4,172.150650,159
5,172.195129,159
6,166.710052,159
7,170.995911,159
8,170.997040,159
9,170.998138,159


In [351]:
ar1 = sample['Actual'].unique()
results = pd.DataFrame()
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [352]:
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)
results.reset_index(drop = True)
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

34.285714285714285
